# Few Shot Notebook

In [1]:
import albumentations as A

In [2]:
import os
import shutil
import time
import pprint

import torch
import torch.nn as nn
import torch.autograd.variable as Variable

from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t
import numpy as np
from collections import OrderedDict

class GaussianNoise(nn.Module):
    def __init__(self, batch_size, input_shape=(3, 84, 84), std=0.05):
        super(GaussianNoise, self).__init__()
        self.shape = (batch_size,) + input_shape
        self.noise = Variable(torch.zeros(self.shape).cuda())
        self.std = std

    def forward(self, x, std=0.15):
        noise = Variable(torch.zeros(x.shape).cuda())
        noise = noise.data.normal_(0, std=std)
        return x + noise


def set_gpu(x):
    os.environ['CUDA_VISIBLE_DEVICES'] = x
    print('using gpu:', x)


def clone(tensor):
    """Detach and clone a tensor including the ``requires_grad`` attribute.

    Arguments:
        tensor (torch.Tensor): tensor to clone.
    """
    cloned = tensor.clone()#tensor.detach().clone()
    # cloned.requires_grad = tensor.requires_grad
    # if tensor.grad is not None:
    #     cloned.grad = clone(tensor.grad)
    return cloned

def clone_state_dict(state_dict):
    """Clone a state_dict. If state_dict is from a ``torch.nn.Module``, use ``keep_vars=True``.

    Arguments:
        state_dict (OrderedDict): the state_dict to clone. Assumes state_dict is not detached from model state.
    """
    return OrderedDict([(name, clone(param)) for name, param in state_dict.items()])

def ensure_path(path):
    if os.path.exists(path):
        if input('{} exists, remove? ([y]/n)'.format(path)) != 'n':
            shutil.rmtree(path)
            os.mkdir(path)
    else:
        os.mkdir(path)

class Averager():
    def __init__(self):
        self.n = 0
        self.v = 0

    def add(self, x):
        self.v = (self.v * self.n + x) / (self.n + 1)
        self.n += 1

    def item(self):
        return self.v

def count_acc(logits, label):
    pred = torch.argmax(logits, dim=1)
    return (pred == label).type(torch.cuda.FloatTensor).mean().item()

def dot_metric(a, b):
    return torch.mm(a, b.t())

def euclidean_metric(a, b):
    n = a.shape[0]
    m = b.shape[0]
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    logits = -((a - b)**2).sum(dim=2)
    return logits

class Timer():

    def __init__(self):
        self.o = time.time()

    def measure(self, p=1):
        x = (time.time() - self.o) / p
        x = int(x)
        if x >= 3600:
            return '{:.1f}h'.format(x / 3600)
        if x >= 60:
            return '{}m'.format(round(x / 60))
        return '{}s'.format(x)

_utils_pp = pprint.PrettyPrinter()
def pprint(x):
    _utils_pp.pprint(x)

def l2_loss(pred, label):
    return ((pred - label)**2).sum() / len(pred) / 2

def set_protocol(data_path, protocol, test_protocol, subset=None):
    train = []
    val = []
    all_set = ['shn', 'hon', 'clv', 'clk', 'gls', 'scl', 'sci', 'nat', 'shx', 'rel']
    if subset is not None:
        train.append(data_path + '/crops_' + subset + '/')
        val.append(data_path + '/crops_' + subset + '/')
    if protocol == 'p1':
        for i in range(3):
            train.append(data_path + '/crops_' + all_set[i])
    elif protocol == 'p2':
        for i in range(3, 6):
            train.append(data_path + '/crops_' + all_set[i])
    elif protocol == 'p3':
        for i in range(6, 8):
            train.append(data_path + '/crops_' + all_set[i])
    elif protocol == 'p4':
        for i in range(8, 10):
            train.append(data_path + '/crops_' + all_set[i])

    if test_protocol == 'p1':
        for i in range(3):
            val.append(data_path + '/crops_' + all_set[i])
    elif test_protocol == 'p2':
        for i in range(3, 6):
            val.append(data_path + '/crops_' + all_set[i])
    elif test_protocol == 'p3':
        for i in range(6, 8):
            val.append(data_path + '/crops_' + all_set[i])
    elif test_protocol == 'p4':
        for i in range(8, 10):
            val.append(data_path + '/crops_' + all_set[i])
    return train, val

def flip(x, dim):
    dim = x.dim() + dim if dim < 0 else dim
    return x[tuple(slice(None, None) if i != dim
             else torch.arange(x.size(i)-1, -1, -1).long()
             for i in range(x.dim()))]

def perturb(data):
    randno = np.random.randint(0, 5)
    if randno == 1:
        return torch.cat((data, data.flip(3)), dim=0)
    elif randno == 2: #180
        return torch.cat((data, data.flip(2)), dim=0)
    elif randno == 3: #90
        return torch.cat((data, data.transpose(2,3)), dim=0)
    else:
        return torch.cat((data, data.transpose(2, 3).flip(3)), dim=0)

## Datamanager

In [3]:
import os.path as osp
from PIL import Image
import numpy as np
import itertools as it

from torch.utils.data import Dataset
from torchvision import transforms

ROOT_PATH = './materials/'

def get_ROI(I):
    row = []
    col = []
    for i in range(I.shape[0]):
        if np.sum(I[i])!=0:
            row+=[i]
    for i in range(I.shape[1]):
        if np.sum(I[:,i])!=0:
            col+=[i]
    return row,col

class Jantung(Dataset):

    def __init__(self, setname, img_path):
        csv_path = osp.join(ROOT_PATH, setname + '_jantung_random3.csv')
        lines = [x.strip() for x in open(csv_path, 'r').readlines()][1:]
        IMG_PATH = img_path
        data = []
        label = []
        lb = -1

        self.wnids = []

        for l in lines:
            wnid = l.split(',')[-1]
            paths = ()
            for n in l.split(',')[:-1]:
                paths += (IMG_PATH + '/' + n,)
            if wnid not in self.wnids:
                self.wnids.append(wnid)
                lb += 1
            data.append(paths)
            label.append(lb)

        self.data = data
        self.label = label

        self.prim = {'6': [14, 26, 38, 49, 59, 69],
                     '5': [15, 29, 51, 67, 93],
                     '4': [16, 51, 85, 103],
                     '3': [21, 87, 147],
                     '2': [85, 170]}[str(len(data[0])//2)]

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        path, label = self.data[i], self.label[i]
        img_tupple = ()
        side = {}
        for diastol in ['ED','ES']:
            for sl in range(len(path)):
                if diastol in path[sl]:
                    try:
                        img        = Image.open(path[sl]).convert('L')
                        img        = np.array(img)
                    except:
                        img        = Image.open(path[sl+4]).convert('L')
                        img        = np.array(img)
                        x_shape    = img.shape
                        img        = np.zeros(x_shape)
                    # replace mask value
                    img[img==85] = 1
                    img[img==170] = 2
                    img[img==255] = 3
                    imgs = [(img == v) for v in range(4)]
                    img = np.stack(imgs, axis=-1).astype('float')
                    img[...,1] = np.where(img[...,1]==1,self.prim[sl//2],0) # red
                    img[...,2] = np.where(img[...,2]==1,self.prim[sl//2],0) # green
                    img[...,3] = np.where(img[...,3]==1,self.prim[sl//2],0) # blue
                    img_tupple += (img[...,1:],)
            img     = np.sum(img_tupple, axis=0)
            img     = Image.fromarray(np.uint8(img))
            img     = np.asarray(img)
            row_a   = min(get_ROI(img)[0])
            col_a   = min(get_ROI(img)[1])
            row_b   = max(get_ROI(img)[0])
            col_b   = max(get_ROI(img)[1])

            res = (row_b-row_a) - (col_b-col_a)
            if res<0:
                row_a-=abs(res)//2
                row_b+=abs(res)//2
                row_b+=abs(res)%2
            else:
                col_a-=abs(res)//2
                col_b+=abs(res)//2
                col_b+=abs(res)%2

            img     = img[row_a:row_b,col_a:col_b]
            side[diastol] = img
        img = np.zeros([side['ED'].shape[0]+side['ES'].shape[0],max(side['ED'].shape[1],side['ES'].shape[1]),3])
        img[:side['ED'].shape[0],:side['ED'].shape[1],:] = side['ED']
        img[side['ED'].shape[0]:,:side['ES'].shape[0],:] = side['ES']
        img     = Image.fromarray(np.uint8(img))
        img     = img.resize((64, 64*2))

        image   = self.transform(img)
        
        return image, label

def hard_augmentation():
    return A.Compose([
        A.OneOf([
            A.Sharpen(p=1),
            A.MotionBlur(p=1),
            A.MedianBlur(blur_limit=3, p=1),
            A.Blur(blur_limit=3, p=1),
        ], p=0.4),
        A.Flip(p=.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=.5),
        A.OneOf([           
            A.GaussNoise(),
            A.OpticalDistortion(p=0.2),
            A.GridDistortion(p=.2),                    
        ], p=0.3),
        A.OneOf([
            A.CLAHE(),
            A.RandomContrast(),
            A.RandomBrightness(),
            A.RandomGamma(),
            A.HueSaturationValue(),
        ], p=0.5),
        A.Cutout(num_holes=8, max_h_size=4, max_w_size=4, fill_value=0, p=0.2),
        A.Cutout(num_holes=4, max_h_size=8, max_w_size=8, fill_value=0, p=0.3),
    ],p=1.0)

class JantungAug(Dataset):

    def __init__(self, setname, img_path):
        csv_path = osp.join(ROOT_PATH, setname + '_jantung_random3.csv')
        lines = [x.strip() for x in open(csv_path, 'r').readlines()][1:]
        IMG_PATH = img_path
        data = []
        label = []
        lb = -1

        self.wnids = []

        for l in lines:
            wnid = l.split(',')[-1]
            paths = ()
            for n in l.split(',')[:-1]:
                paths += (IMG_PATH + '/' + n,)
            if wnid not in self.wnids:
                self.wnids.append(wnid)
                lb += 1
            data.append(paths)
            label.append(lb)

        self.data = data
        self.label = label

        self.prim = {'6': [14, 26, 38, 49, 59, 69],
                     '5': [15, 29, 51, 67, 93],
                     '4': [16, 51, 85, 103],
                     '3': [21, 87, 147],
                     '2': [85, 170]}[str(len(data[0])//2)]
        
        self.augmentation = hard_augmentation()

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        path, label = self.data[i], self.label[i]
        img_tupple = ()
        side = {}
        for diastol in ['ED','ES']:
            for sl in range(len(path)):
                if diastol in path[sl]:
                    try:
                        img        = Image.open(path[sl]).convert('L')
                        img        = np.array(img)
                    except:
                        img        = Image.open(path[sl+4]).convert('L')
                        img        = np.array(img)
                        x_shape    = img.shape
                        img        = np.zeros(x_shape)
                    # replace mask value
                    img[img==85] = 1
                    img[img==170] = 2
                    img[img==255] = 3
                    imgs = [(img == v) for v in range(4)]
                    img = np.stack(imgs, axis=-1).astype('float')
                    img[...,1] = np.where(img[...,1]==1,self.prim[sl//2],0) # 
                    img[...,2] = np.where(img[...,2]==1,self.prim[sl//2],0) #
                    img[...,3] = np.where(img[...,3]==1,self.prim[sl//2],0) # blue
                    img_tupple += (img[...,1:],)
            img     = np.sum(img_tupple, axis=0)
            img     = Image.fromarray(np.uint8(img))
            img     = np.asarray(img)
            row_a   = min(get_ROI(img)[0])
            col_a   = min(get_ROI(img)[1])
            row_b   = max(get_ROI(img)[0])
            col_b   = max(get_ROI(img)[1])

            res = (row_b-row_a) - (col_b-col_a)
            if res<0:
                row_a-=abs(res)//2
                row_b+=abs(res)//2
                row_b+=abs(res)%2
            else:
                col_a-=abs(res)//2
                col_b+=abs(res)//2
                col_b+=abs(res)%2

            img     = img[row_a:row_b,col_a:col_b]
            side[diastol] = img
        img = np.zeros([side['ED'].shape[0]+side['ES'].shape[0],max(side['ED'].shape[1],side['ES'].shape[1]),3])
        img[:side['ED'].shape[0],:side['ED'].shape[1],:] = side['ED']
        img[side['ED'].shape[0]:,:side['ES'].shape[0],:] = side['ES']
        img     = self.augmentation(image=np.uint8(img))['image']
        img     = Image.fromarray(np.uint8(img))
        img     = img.resize((64, 64*2))

        image   = self.transform(img)

        return image, label

## Sampler

In [4]:
import torch
import numpy as np


class CategoriesSampler():

    def __init__(self, label, n_batch, n_cls, n_per):
        self.n_batch = n_batch
        self.n_cls = n_cls
        self.n_per = n_per

        label = np.array(label)
        self.m_ind = []
        for i in range(max(label) + 1):
            ind = np.argwhere(label == i).reshape(-1)
            ind = torch.from_numpy(ind)
            if len(ind) > 2:
                self.m_ind.append(ind)

    def __len__(self):
        return self.n_batch
    
    def __iter__(self):
        for i_batch in range(self.n_batch):
            batch = []
            classes = torch.randperm(len(self.m_ind))[:self.n_cls]
            for c in classes:
                l = self.m_ind[c]
                pos = torch.randperm(len(l))[:self.n_per]
                batch.append(l[pos])
            batch = torch.stack(batch).t().reshape(-1)
            #for i in range(1000):
            yield batch


class CategoriesSampler_Custom():

    def __init__(self, label, n_batch, n_cls, n_per):
        self.n_batch = n_batch
        self.n_cls = n_cls
        self.n_per = n_per
        self.n_label = len(label)

    def __len__(self):
        return self.n_batch
    
    def __iter__(self):
        for i_batch in range(self.n_batch):
            batch = []
            if self.n_label!=50:
                """for h in range(100):
                    for k in range(0,self.n_label,100):
                        batch.append(k+h)
                if i_batch%2!=0:
                    batch = batch[100:]+batch[:100]
                batch = torch.tensor(batch)
                yield batch"""
                pass
            else:
                yield torch.randperm(50)


## Convnet

In [5]:
from torchsummary import summary
import torchvision.models as models
import torch.nn as nn
import torch
import numpy as np

def BasicBlock():
    return 0

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        # nn.Hardswish(),
        nn.MaxPool2d(2)
    )

class ConvNet(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(x_dim, hid_dim),
            conv_block(hid_dim, hid_dim),
            conv_block(hid_dim, hid_dim),
            conv_block(hid_dim, z_dim),
        )
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)


from efficientnet_pytorch import EfficientNet
class Efnet_do(nn.Module):

    def __init__(self, ver='b0', x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-'+ver, include_top = False)
        self.drop = nn.Dropout(0.5)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return self.drop(x.view(x.size(0), -1))

class Efnet_nopre(nn.Module):

    def __init__(self, ver='b0', x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = EfficientNet.from_name('efficientnet-'+ver, include_top = False)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)
        
class Efnet(nn.Module):

    def __init__(self, ver='b0', x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-'+ver, include_top = False)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

        
import torchvision.models as models
class Mbnet_nopre(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = models.mobilenet_v3_large(pretrained=False)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

class Mbnet(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = models.mobilenet_v3_large(pretrained=True)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)


## Classifier

In [6]:
import torch
import torch.nn as nn

class Subspace_Projection(nn.Module):
    def __init__(self, num_dim=2):
        super().__init__()
        self.num_dim = num_dim

    def create_subspace(self, supportset_features, class_size, sample_size):
        all_hyper_planes = []
        means = []
        for ii in range(class_size):
            num_sample = sample_size
            all_support_within_class_t = supportset_features[ii]
            meann = torch.mean(all_support_within_class_t, dim=0)
            means.append(meann)
            all_support_within_class_t = all_support_within_class_t - meann.unsqueeze(0).repeat(num_sample, 1)
            all_support_within_class = torch.transpose(all_support_within_class_t, 0, 1)
            # print(all_support_within_class.double())
            uu, s, v = torch.svd(all_support_within_class.double(), some=False)
            uu = uu.float()
            all_hyper_planes.append(uu[:, :self.num_dim])

        all_hyper_planes = torch.stack(all_hyper_planes, dim=0)
        means = torch.stack(means)

        if len(all_hyper_planes.size()) < 3:
            all_hyper_planes = all_hyper_planes.unsqueeze(-1)

        return all_hyper_planes, means


    def projection_metric(self, target_features, hyperplanes, mu):
        eps = 1e-12
        batch_size = target_features.shape[0]
        class_size = hyperplanes.shape[0]

        similarities = []

        discriminative_loss = 0.0

        for j in range(class_size):
            h_plane_j =  hyperplanes[j].unsqueeze(0).repeat(batch_size, 1, 1)
            target_features_expanded = (target_features - mu[j].expand_as(target_features)).unsqueeze(-1)
            projected_query_j = torch.bmm(h_plane_j, torch.bmm(torch.transpose(h_plane_j, 1, 2), target_features_expanded))
            projected_query_j = torch.squeeze(projected_query_j) + mu[j].unsqueeze(0).repeat(batch_size, 1)
            projected_query_dist_inter = target_features - projected_query_j

            #Training per epoch is slower but less epochs in total
            query_loss = -torch.sqrt(torch.sum(projected_query_dist_inter * projected_query_dist_inter, dim=-1) + eps) # norm ||.||

            #Training per epoch is faster but more epochs in total
            #query_loss = -torch.sum(projected_query_dist_inter * projected_query_dist_inter, dim=-1) # Squared norm ||.||^2

            similarities.append(query_loss)

            for k in range(class_size):
                if j != k:
                   temp_loss = torch.mm(torch.transpose(hyperplanes[j], 0, 1), hyperplanes[k]) ## discriminative subspaces (Conv4 only, ResNet12 is computationally expensive)
                   discriminative_loss = discriminative_loss + torch.sum(temp_loss*temp_loss)

        similarities = torch.stack(similarities, dim=1)

        return similarities, discriminative_loss


## Training

In [7]:
import os
import torch
import argparse
import numpy as np
import os.path as osp
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader

# SET MODEL NAME
# modelname = 'cnn'
# modelname = 'mbnet'
# modelname = 'mbnet_no'
# modelname = 'efnetb1'
# modelname = 'efnetb1_no'
# modelname = 'efnetb0'
# modelname = 'efnetb2'
# modelname = 'efnetb3'
# modelname = 'efnetb1_do'
# modelname = 'efnetb1aug'
modelname = 'efnetb1_doaug'

num_sampler = 100
nslice = 6

txt = str(datetime.now())
txt = '_'.join([modelname+'-final', txt[:4], txt[5:7], txt[8:10], txt[11:13], txt[14:16]])

args = {}
args['max-epoch'] = 10
args['save-epoch'] = 5
args['shot'] = 5
args['query'] = 5
args['train-way'] = 5
args['test-way'] = 5
args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final-shot5/'+txt
args['data-path'] = 'materials'
args['gpu'] = '0'
args['lamb'] = 0.03
args['subspace-dim'] = args['shot']-1
set_gpu(args['gpu'])

# MODEL BUILDER
model = {
    'cnn': ConvNet(),
    'mbnet_no': Mbnet_nopre(),
    'mbnet': Mbnet(),
    'efnetb1_no': Efnet_nopre(ver='b1'),
    'efnetb0': Efnet(ver='b0'),
    'efnetb1': Efnet(ver='b1'),
    'efnetb2': Efnet(ver='b2'),
    'efnetb3': Efnet(ver='b3'),
    'efnetb1_do': Efnet_do(ver='b1'),
}[modelname.replace('aug','')].cuda()


using gpu: 0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b1


In [8]:
# GENERATE RANDOM TRAINING VALIDATION
train = []
test = []
csvpath = 'materials/'
csvfile = [_[:-1] for _ in open(csvpath+'all_random3.csv', 'r').readlines()[1:]]
dcm,hcm,minf,nor,rv = np.arange(0,20), np.arange(20,40), np.arange(40,60), np.arange(60,80), np.arange(80,100)
for _ in [dcm,hcm,minf,nor,rv]:
    perm = list(np.random.permutation(_))
    train += [csvfile[k] for k in perm[:10]] 
    test += [csvfile[k] for k in perm[10:]]
    
test = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in test]
train = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in train]

f =  open(csvpath+'train_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in train[:-1]:
    f.write(wr+'\n')
f.write(train[-1])
f.close()

f =  open(csvpath+'test_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in test[:-1]:
    f.write(wr+'\n')
f.write(test[-1])
f.close()

In [9]:
# DATA LOADER
if 'aug' in modelname:
  trainset = JantungAug('train', args['data-path'])
else:
  trainset = Jantung('train', args['data-path'])
  
train_sampler = CategoriesSampler(trainset.label, num_sampler,
                                  args['train-way'], args['shot'] + args['query'])
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, shuffle=False,)
#                           num_workers=8, pin_memory=True)

valset = Jantung('test', args['data-path'])
val_sampler = CategoriesSampler(valset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])
list_val_sampler = []
for pr in val_sampler:
    list_val_sampler.append([h.item() for h in pr])
list_val_sampler = torch.from_numpy(np.array(list_val_sampler))
val_loader = DataLoader(dataset=valset, batch_sampler=list_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

testset = Jantung('testfinal', args['data-path'])
test_sampler = CategoriesSampler_Custom(testset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])

test_loader = DataLoader(dataset=testset, batch_sampler=test_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)


C:\Users\User\anaconda3\envs\torch\lib\site-packages\albumentations\augmentations\transforms.py:1725: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\Users\User\anaconda3\envs\torch\lib\site-packages\albumentations\augmentations\transforms.py:1699: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\Users\User\anaconda3\envs\torch\lib\site-packages\albumentations\augmentations\transforms.py:643: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [10]:
from tqdm import tqdm
projection_pro = Subspace_Projection(num_dim=args['subspace-dim'])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if args['shot'] > 1:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

if args['shot'] == 1:
    shot_num = 2
    args['subspace-dim'] = 1
else:
    shot_num = args['shot']

In [11]:
# TRAINING PHASE
if not os.path.exists(args['save-path']):
    os.mkdir(args['save-path'])
torch.save(list_val_sampler, args['save-path']+'/support-set.pt')
def save_model(name):
    if not os.path.exists(args['save-path']):
        os.mkdir(args['save-path'])
    torch.save(model.state_dict(), osp.join(args['save-path'], name + '.pth'))

trlog = {}
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0
trlog['max_epoch'] = 0

timer = Timer()

for epoch in range(1, args['max-epoch'] + 1):
    lr_scheduler.step()

    model.train()

    tl = Averager()
    ta = Averager()

    for i, batch in tqdm(enumerate(train_loader, 1)):
        data, _ = [_.cuda() for _ in batch]
        p = args['shot'] * args['train-way']
        qq = p + args['query'] * args['train-way']
        data_shot, data_query = data[:p], data[p:qq]
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['train-way'], -1)
        proto = torch.transpose(proto, 0, 1)
        hyperplanes, mu = projection_pro.create_subspace(proto, args['train-way'], shot_num)

        label = torch.arange(args['train-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, discriminative_loss = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        loss = F.cross_entropy(logits, label) + args['lamb']*discriminative_loss
        acc = count_acc(logits, label)

        tl.add(loss.item())
        ta.add(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch {},   loss={:.4f} acc={:.4f}'
          .format(epoch,     loss.item(), acc))

    tl = tl.item()
    ta = ta.item()

    if epoch < 100 and epoch%2!=0:
        continue
    model.eval()

    vl = Averager()
    va = Averager()

    vk = []
    vs = []
    for pr in list_val_sampler:
        vk.append([valset.label[h.item()] for h in pr])
        vs.append([h.item() for h in pr])
        
    val_support_data = [] 
    val_support_label = [] 
    for i, batch in tqdm(enumerate(val_loader, 1)):
        data, _ = [_.cuda() for _ in batch] 
        
        p = args['shot'] * args['test-way']
        
        data_shot, data_query = data[:p], data[p:]

        val_support_data.append(data_shot) 
        val_support_label.append(_[:p][:5])        
        print('data +=',vs[i-1][p:])
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['test-way'], -1) 
        proto = torch.transpose(proto, 0, 1)
        hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
        
        label = torch.arange(args['test-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        prediction = torch.argmax(logits, axis=1)
        pl = vk[i-1][p:][:5]
        pred_label = [pl[ii.item()] for ii in prediction]
        print('pred_label +=',pred_label)

        pros = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()==label[ii].item()]
        cons = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()!=label[ii].item()]
        print('pred +=',cons)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)

        vl.add(loss.item())
        va.add(acc)

    vl = vl.item()
    va = va.item()
    print('epoch {}, val, loss={:.4f} acc={:.4f} maxacc={:.4f}'.format(epoch, vl, va,trlog['max_acc']))

    if va > trlog['max_acc']:
        trlog['max_acc'] = va
        save_model('max-acc')
        trlog['max_epoch'] = epoch

    trlog['train_loss'].append(tl)
    trlog['train_acc'].append(ta)
    trlog['val_loss'].append(vl)
    trlog['val_acc'].append(va)

    torch.save(trlog, osp.join(args['save-path'], 'trlog'))

    save_model('epoch-last')

    if epoch % args['save-epoch'] == 0:
        save_model('epoch-{}'.format(epoch))

    print('ETA:{}/{}'.format(timer.measure(), timer.measure(epoch / args['max-epoch'])))


C:\Users\User\anaconda3\envs\torch\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100it [07:17,  4.37s/it]
0it [00:00, ?it/s]

epoch 1,   loss=0.6042 acc=0.8400


100it [07:12,  4.33s/it]
0it [00:00, ?it/s]

epoch 2,   loss=0.2581 acc=0.9200


1it [00:05,  5.22s/it]

...

99it [07:03,  4.28s/it]

data += [29, 9, 19, 37, 42, 27, 6, 15, 30, 40, 25, 0, 13, 33, 46, 23, 1, 16, 38, 43, 20, 3, 18, 35, 45]
pred_label += [0, 0, 3, 3, 4, 0, 0, 1, 4, 4, 1, 0, 1, 3, 4, 0, 0, 1, 3, 4, 2, 0, 1, 3, 3]
pred += [29, 19, 27, 30, 25, 23, 45]


100it [07:07,  4.28s/it]

data += [17, 20, 7, 45, 32, 16, 27, 1, 44, 30, 13, 22, 5, 42, 37, 15, 26, 4, 41, 35, 11, 21, 9, 46, 38]
pred_label += [1, 3, 0, 3, 3, 1, 0, 0, 4, 4, 1, 3, 0, 4, 3, 1, 2, 0, 4, 3, 1, 0, 0, 4, 3]
pred += [20, 45, 27, 30, 22, 21]
epoch 10, val, loss=1.7743 acc=0.7084 maxacc=0.7288


ETA:1.8h/1.8h


## Testing

In [12]:
# os.listdir('./save/subspace-5w5sdiscriminativeJantung_final/')

In [13]:
# args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final/efnetb1_doaug-final_2021_10_07_08_06'

In [14]:
print("-----------------------------TESTING-----------------------------")
print("-----------------------------------------------------------------")

from tqdm import tqdm

# LOAD SUPPORT SET
trlog = torch.load(args['save-path']+'/trlog')
support_val_sampler = torch.load(args['save-path']+'/support-set.pt')
val_loader = DataLoader(dataset=valset, batch_sampler=support_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

val_support_data = [] 
val_support_label = [] 
for i, batch in tqdm(enumerate(val_loader, 1)):
    data, _ = [_.cuda() for _ in batch] 
    p = args['shot'] * args['test-way']
    data_shot, data_query = data[:p], data[p:]
    val_support_data.append(data_shot) 
    val_support_label.append(_[:p][:5]) 

# LOAD MODEL
model.load_state_dict(torch.load(osp.join(args['save-path'], 'max-acc.pth')))
model.eval()

trlog['test_data_in_max_acc'] = []
trlog['test_label_in_max_acc'] = []

for i, batch in tqdm(enumerate(test_loader, 1)):
    data, _ = [_.cuda() for _ in batch]

    p = args['shot'] * args['test-way'] 
    data_shot, data_query = data[:p], data[p:]
    data_shot = val_support_data[i-1] 

    if args['shot'] == 1:
        data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

    proto = model(data_shot)
    proto = proto.reshape(shot_num, args['test-way'], -1) 
    proto = torch.transpose(proto, 0, 1)
    hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)

    logits, ___ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
    prediction = torch.argmax(logits, axis=1)
    prediction = [val_support_label[i-1][il.item()].item() for il in prediction]
    
    trlog['test_data_in_max_acc'] += [__.item() for __ in _[p:]] 
    trlog['test_label_in_max_acc'] += prediction

0it [00:00, ?it/s]

-----------------------------TESTING-----------------------------
-----------------------------------------------------------------


100it [07:00,  4.20s/it]
100it [08:52,  5.33s/it]


In [15]:
import pandas as pd

pd.DataFrame({
    'data':trlog['test_data_in_max_acc'],
    'prediction':trlog['test_label_in_max_acc']
    }).to_csv(
        args['save-path']+'/{}-finaltest-epochs{}_valacc{}.csv'.format(
            modelname,
            trlog['max_epoch'],
            int(round(trlog['max_acc'],4)*10000)), 
        index=False)  

## Training

In [16]:
import os
import torch
import argparse
import numpy as np
import os.path as osp
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader

# SET MODEL NAME
# modelname = 'cnn'
# modelname = 'mbnet'
# modelname = 'mbnet_no'
# modelname = 'efnetb1'
# modelname = 'efnetb1_no'
# modelname = 'efnetb0'
# modelname = 'efnetb2'
# modelname = 'efnetb3'
# modelname = 'efnetb1_do'
# modelname = 'efnetb1aug'
modelname = 'efnetb1_doaug'

num_sampler = 100
nslice = 6

txt = str(datetime.now())
txt = '_'.join([modelname+'-final', txt[:4], txt[5:7], txt[8:10], txt[11:13], txt[14:16]])

args = {}
args['max-epoch'] = 10
args['save-epoch'] = 5
args['shot'] = 5
args['query'] = 5
args['train-way'] = 5
args['test-way'] = 5
args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final-shot5/'+txt
args['data-path'] = 'materials'
args['gpu'] = '0'
args['lamb'] = 0.03
args['subspace-dim'] = args['shot']-1
set_gpu(args['gpu'])

# MODEL BUILDER
model = {
    'cnn': ConvNet(),
    'mbnet_no': Mbnet_nopre(),
    'mbnet': Mbnet(),
    'efnetb1_no': Efnet_nopre(ver='b1'),
    'efnetb0': Efnet(ver='b0'),
    'efnetb1': Efnet(ver='b1'),
    'efnetb2': Efnet(ver='b2'),
    'efnetb3': Efnet(ver='b3'),
    'efnetb1_do': Efnet_do(ver='b1'),
}[modelname.replace('aug','')].cuda()


using gpu: 0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b1


In [17]:
# GENERATE RANDOM TRAINING VALIDATION
train = []
test = []
csvpath = 'materials/'
csvfile = [_[:-1] for _ in open(csvpath+'all_random3.csv', 'r').readlines()[1:]]
dcm,hcm,minf,nor,rv = np.arange(0,20), np.arange(20,40), np.arange(40,60), np.arange(60,80), np.arange(80,100)
for _ in [dcm,hcm,minf,nor,rv]:
    perm = list(np.random.permutation(_))
    train += [csvfile[k] for k in perm[:10]] 
    test += [csvfile[k] for k in perm[10:]]
    
test = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in test]
train = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in train]

f =  open(csvpath+'train_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in train[:-1]:
    f.write(wr+'\n')
f.write(train[-1])
f.close()

f =  open(csvpath+'test_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in test[:-1]:
    f.write(wr+'\n')
f.write(test[-1])
f.close()

In [18]:
# DATA LOADER
if 'aug' in modelname:
  trainset = JantungAug('train', args['data-path'])
else:
  trainset = Jantung('train', args['data-path'])
  
train_sampler = CategoriesSampler(trainset.label, num_sampler,
                                  args['train-way'], args['shot'] + args['query'])
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, shuffle=False,)
#                           num_workers=8, pin_memory=True)

valset = Jantung('test', args['data-path'])
val_sampler = CategoriesSampler(valset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])
list_val_sampler = []
for pr in val_sampler:
    list_val_sampler.append([h.item() for h in pr])
list_val_sampler = torch.from_numpy(np.array(list_val_sampler))
val_loader = DataLoader(dataset=valset, batch_sampler=list_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

testset = Jantung('testfinal', args['data-path'])
test_sampler = CategoriesSampler_Custom(testset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])

test_loader = DataLoader(dataset=testset, batch_sampler=test_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)


In [19]:
from tqdm import tqdm
projection_pro = Subspace_Projection(num_dim=args['subspace-dim'])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if args['shot'] > 1:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

if args['shot'] == 1:
    shot_num = 2
    args['subspace-dim'] = 1
else:
    shot_num = args['shot']

In [20]:
# TRAINING PHASE
if not os.path.exists(args['save-path']):
    os.mkdir(args['save-path'])
torch.save(list_val_sampler, args['save-path']+'/support-set.pt')
def save_model(name):
    if not os.path.exists(args['save-path']):
        os.mkdir(args['save-path'])
    torch.save(model.state_dict(), osp.join(args['save-path'], name + '.pth'))

trlog = {}
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0
trlog['max_epoch'] = 0

timer = Timer()

for epoch in range(1, args['max-epoch'] + 1):
    lr_scheduler.step()

    model.train()

    tl = Averager()
    ta = Averager()

    for i, batch in tqdm(enumerate(train_loader, 1)):
        data, _ = [_.cuda() for _ in batch]
        p = args['shot'] * args['train-way']
        qq = p + args['query'] * args['train-way']
        data_shot, data_query = data[:p], data[p:qq]
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['train-way'], -1)
        proto = torch.transpose(proto, 0, 1)
        hyperplanes, mu = projection_pro.create_subspace(proto, args['train-way'], shot_num)

        label = torch.arange(args['train-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, discriminative_loss = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        loss = F.cross_entropy(logits, label) + args['lamb']*discriminative_loss
        acc = count_acc(logits, label)

        tl.add(loss.item())
        ta.add(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch {},   loss={:.4f} acc={:.4f}'
          .format(epoch,     loss.item(), acc))

    tl = tl.item()
    ta = ta.item()

    if epoch < 100 and epoch%2!=0:
        continue
    model.eval()

    vl = Averager()
    va = Averager()

    vk = []
    vs = []
    for pr in list_val_sampler:
        vk.append([valset.label[h.item()] for h in pr])
        vs.append([h.item() for h in pr])
        
    val_support_data = [] 
    val_support_label = [] 
    for i, batch in tqdm(enumerate(val_loader, 1)):
        data, _ = [_.cuda() for _ in batch] 
        
        p = args['shot'] * args['test-way']
        
        data_shot, data_query = data[:p], data[p:]

        val_support_data.append(data_shot) 
        val_support_label.append(_[:p][:5])        
        print('data +=',vs[i-1][p:])
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['test-way'], -1) 
        proto = torch.transpose(proto, 0, 1)
        hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
        
        label = torch.arange(args['test-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        prediction = torch.argmax(logits, axis=1)
        pl = vk[i-1][p:][:5]
        pred_label = [pl[ii.item()] for ii in prediction]
        print('pred_label +=',pred_label)

        pros = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()==label[ii].item()]
        cons = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()!=label[ii].item()]
        print('pred +=',cons)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)

        vl.add(loss.item())
        va.add(acc)

    vl = vl.item()
    va = va.item()
    print('epoch {}, val, loss={:.4f} acc={:.4f} maxacc={:.4f}'.format(epoch, vl, va,trlog['max_acc']))

    if va > trlog['max_acc']:
        trlog['max_acc'] = va
        save_model('max-acc')
        trlog['max_epoch'] = epoch

    trlog['train_loss'].append(tl)
    trlog['train_acc'].append(ta)
    trlog['val_loss'].append(vl)
    trlog['val_acc'].append(va)

    torch.save(trlog, osp.join(args['save-path'], 'trlog'))

    save_model('epoch-last')

    if epoch % args['save-epoch'] == 0:
        save_model('epoch-{}'.format(epoch))

    print('ETA:{}/{}'.format(timer.measure(), timer.measure(epoch / args['max-epoch'])))


100it [07:24,  4.45s/it]
0it [00:00, ?it/s]

epoch 1,   loss=0.2940 acc=0.8400


100it [07:22,  4.43s/it]
0it [00:00, ?it/s]

epoch 2,   loss=0.5543 acc=0.7200


1it [00:06,  6.35s/it]

...

99it [07:09,  4.34s/it]

data += [32, 45, 0, 19, 22, 31, 43, 6, 18, 25, 30, 48, 8, 16, 23, 36, 44, 3, 13, 26, 38, 46, 9, 10, 27]
pred_label += [3, 4, 0, 1, 3, 3, 4, 0, 4, 1, 4, 4, 2, 1, 2, 3, 4, 0, 1, 2, 3, 4, 2, 1, 0]
pred += [22, 18, 25, 30, 8, 9, 27]


100it [07:13,  4.34s/it]

data += [13, 26, 45, 33, 1, 19, 21, 44, 34, 0, 15, 25, 43, 36, 2, 14, 29, 42, 39, 7, 16, 20, 41, 30, 5]
pred_label += [1, 0, 4, 3, 0, 1, 2, 4, 3, 0, 1, 1, 4, 3, 2, 1, 0, 4, 3, 2, 1, 2, 3, 4, 2]
pred += [26, 25, 2, 29, 7, 41, 30, 5]
epoch 10, val, loss=2.1406 acc=0.7172 maxacc=0.7984


ETA:1.8h/1.8h


## Testing

In [21]:
# os.listdir('./save/subspace-5w5sdiscriminativeJantung_final/')

In [22]:
# args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final/efnetb1_doaug-final_2021_10_07_08_06'

In [23]:
print("-----------------------------TESTING-----------------------------")
print("-----------------------------------------------------------------")

from tqdm import tqdm

# LOAD SUPPORT SET
trlog = torch.load(args['save-path']+'/trlog')
support_val_sampler = torch.load(args['save-path']+'/support-set.pt')
val_loader = DataLoader(dataset=valset, batch_sampler=support_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

val_support_data = [] 
val_support_label = [] 
for i, batch in tqdm(enumerate(val_loader, 1)):
    data, _ = [_.cuda() for _ in batch] 
    p = args['shot'] * args['test-way']
    data_shot, data_query = data[:p], data[p:]
    val_support_data.append(data_shot) 
    val_support_label.append(_[:p][:5]) 

# LOAD MODEL
model.load_state_dict(torch.load(osp.join(args['save-path'], 'max-acc.pth')))
model.eval()

trlog['test_data_in_max_acc'] = []
trlog['test_label_in_max_acc'] = []

for i, batch in tqdm(enumerate(test_loader, 1)):
    data, _ = [_.cuda() for _ in batch]

    p = args['shot'] * args['test-way'] 
    data_shot, data_query = data[:p], data[p:]
    data_shot = val_support_data[i-1] 

    if args['shot'] == 1:
        data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

    proto = model(data_shot)
    proto = proto.reshape(shot_num, args['test-way'], -1) 
    proto = torch.transpose(proto, 0, 1)
    hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)

    logits, ___ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
    prediction = torch.argmax(logits, axis=1)
    prediction = [val_support_label[i-1][il.item()].item() for il in prediction]
    
    trlog['test_data_in_max_acc'] += [__.item() for __ in _[p:]] 
    trlog['test_label_in_max_acc'] += prediction

0it [00:00, ?it/s]

-----------------------------TESTING-----------------------------
-----------------------------------------------------------------


100it [07:03,  4.23s/it]
100it [09:01,  5.41s/it]


In [24]:
import pandas as pd

pd.DataFrame({
    'data':trlog['test_data_in_max_acc'],
    'prediction':trlog['test_label_in_max_acc']
    }).to_csv(
        args['save-path']+'/{}-finaltest-epochs{}_valacc{}.csv'.format(
            modelname,
            trlog['max_epoch'],
            int(round(trlog['max_acc'],4)*10000)), 
        index=False)  

## Training

In [25]:
import os
import torch
import argparse
import numpy as np
import os.path as osp
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader

# SET MODEL NAME
# modelname = 'cnn'
# modelname = 'mbnet'
# modelname = 'mbnet_no'
# modelname = 'efnetb1'
# modelname = 'efnetb1_no'
# modelname = 'efnetb0'
# modelname = 'efnetb2'
# modelname = 'efnetb3'
# modelname = 'efnetb1_do'
# modelname = 'efnetb1aug'
modelname = 'efnetb1_doaug'

num_sampler = 100
nslice = 6

txt = str(datetime.now())
txt = '_'.join([modelname+'-final', txt[:4], txt[5:7], txt[8:10], txt[11:13], txt[14:16]])

args = {}
args['max-epoch'] = 10
args['save-epoch'] = 5
args['shot'] = 5
args['query'] = 5
args['train-way'] = 5
args['test-way'] = 5
args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final-shot5/'+txt
args['data-path'] = 'materials'
args['gpu'] = '0'
args['lamb'] = 0.03
args['subspace-dim'] = args['shot']-1
set_gpu(args['gpu'])

# MODEL BUILDER
model = {
    'cnn': ConvNet(),
    'mbnet_no': Mbnet_nopre(),
    'mbnet': Mbnet(),
    'efnetb1_no': Efnet_nopre(ver='b1'),
    'efnetb0': Efnet(ver='b0'),
    'efnetb1': Efnet(ver='b1'),
    'efnetb2': Efnet(ver='b2'),
    'efnetb3': Efnet(ver='b3'),
    'efnetb1_do': Efnet_do(ver='b1'),
}[modelname.replace('aug','')].cuda()


using gpu: 0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b1


In [26]:
# GENERATE RANDOM TRAINING VALIDATION
train = []
test = []
csvpath = 'materials/'
csvfile = [_[:-1] for _ in open(csvpath+'all_random3.csv', 'r').readlines()[1:]]
dcm,hcm,minf,nor,rv = np.arange(0,20), np.arange(20,40), np.arange(40,60), np.arange(60,80), np.arange(80,100)
for _ in [dcm,hcm,minf,nor,rv]:
    perm = list(np.random.permutation(_))
    train += [csvfile[k] for k in perm[:10]] 
    test += [csvfile[k] for k in perm[10:]]
    
test = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in test]
train = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in train]

f =  open(csvpath+'train_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in train[:-1]:
    f.write(wr+'\n')
f.write(train[-1])
f.close()

f =  open(csvpath+'test_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in test[:-1]:
    f.write(wr+'\n')
f.write(test[-1])
f.close()

In [27]:
# DATA LOADER
if 'aug' in modelname:
  trainset = JantungAug('train', args['data-path'])
else:
  trainset = Jantung('train', args['data-path'])
  
train_sampler = CategoriesSampler(trainset.label, num_sampler,
                                  args['train-way'], args['shot'] + args['query'])
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, shuffle=False,)
#                           num_workers=8, pin_memory=True)

valset = Jantung('test', args['data-path'])
val_sampler = CategoriesSampler(valset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])
list_val_sampler = []
for pr in val_sampler:
    list_val_sampler.append([h.item() for h in pr])
list_val_sampler = torch.from_numpy(np.array(list_val_sampler))
val_loader = DataLoader(dataset=valset, batch_sampler=list_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

testset = Jantung('testfinal', args['data-path'])
test_sampler = CategoriesSampler_Custom(testset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])

test_loader = DataLoader(dataset=testset, batch_sampler=test_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)


In [28]:
from tqdm import tqdm
projection_pro = Subspace_Projection(num_dim=args['subspace-dim'])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if args['shot'] > 1:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

if args['shot'] == 1:
    shot_num = 2
    args['subspace-dim'] = 1
else:
    shot_num = args['shot']

In [29]:
# TRAINING PHASE
if not os.path.exists(args['save-path']):
    os.mkdir(args['save-path'])
torch.save(list_val_sampler, args['save-path']+'/support-set.pt')
def save_model(name):
    if not os.path.exists(args['save-path']):
        os.mkdir(args['save-path'])
    torch.save(model.state_dict(), osp.join(args['save-path'], name + '.pth'))

trlog = {}
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0
trlog['max_epoch'] = 0

timer = Timer()

for epoch in range(1, args['max-epoch'] + 1):
    lr_scheduler.step()

    model.train()

    tl = Averager()
    ta = Averager()

    for i, batch in tqdm(enumerate(train_loader, 1)):
        data, _ = [_.cuda() for _ in batch]
        p = args['shot'] * args['train-way']
        qq = p + args['query'] * args['train-way']
        data_shot, data_query = data[:p], data[p:qq]
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['train-way'], -1)
        proto = torch.transpose(proto, 0, 1)
        hyperplanes, mu = projection_pro.create_subspace(proto, args['train-way'], shot_num)

        label = torch.arange(args['train-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, discriminative_loss = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        loss = F.cross_entropy(logits, label) + args['lamb']*discriminative_loss
        acc = count_acc(logits, label)

        tl.add(loss.item())
        ta.add(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch {},   loss={:.4f} acc={:.4f}'
          .format(epoch,     loss.item(), acc))

    tl = tl.item()
    ta = ta.item()

    if epoch < 100 and epoch%2!=0:
        continue
    model.eval()

    vl = Averager()
    va = Averager()

    vk = []
    vs = []
    for pr in list_val_sampler:
        vk.append([valset.label[h.item()] for h in pr])
        vs.append([h.item() for h in pr])
        
    val_support_data = [] 
    val_support_label = [] 
    for i, batch in tqdm(enumerate(val_loader, 1)):
        data, _ = [_.cuda() for _ in batch] 
        
        p = args['shot'] * args['test-way']
        
        data_shot, data_query = data[:p], data[p:]

        val_support_data.append(data_shot) 
        val_support_label.append(_[:p][:5])        
        print('data +=',vs[i-1][p:])
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['test-way'], -1) 
        proto = torch.transpose(proto, 0, 1)
        hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
        
        label = torch.arange(args['test-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        prediction = torch.argmax(logits, axis=1)
        pl = vk[i-1][p:][:5]
        pred_label = [pl[ii.item()] for ii in prediction]
        print('pred_label +=',pred_label)

        pros = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()==label[ii].item()]
        cons = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()!=label[ii].item()]
        print('pred +=',cons)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)

        vl.add(loss.item())
        va.add(acc)

    vl = vl.item()
    va = va.item()
    print('epoch {}, val, loss={:.4f} acc={:.4f} maxacc={:.4f}'.format(epoch, vl, va,trlog['max_acc']))

    if va > trlog['max_acc']:
        trlog['max_acc'] = va
        save_model('max-acc')
        trlog['max_epoch'] = epoch

    trlog['train_loss'].append(tl)
    trlog['train_acc'].append(ta)
    trlog['val_loss'].append(vl)
    trlog['val_acc'].append(va)

    torch.save(trlog, osp.join(args['save-path'], 'trlog'))

    save_model('epoch-last')

    if epoch % args['save-epoch'] == 0:
        save_model('epoch-{}'.format(epoch))

    print('ETA:{}/{}'.format(timer.measure(), timer.measure(epoch / args['max-epoch'])))


100it [07:27,  4.47s/it]
0it [00:00, ?it/s]

epoch 1,   loss=0.9213 acc=0.8000


100it [07:28,  4.49s/it]
0it [00:00, ?it/s]

epoch 2,   loss=0.5061 acc=0.9200


1it [00:04,  4.33s/it]

...

99it [07:11,  4.34s/it]

data += [40, 38, 28, 9, 10, 46, 35, 24, 2, 13, 43, 33, 29, 8, 14, 41, 30, 21, 1, 15, 42, 31, 27, 6, 18]
pred_label += [4, 3, 2, 0, 3, 2, 4, 2, 0, 3, 4, 3, 3, 2, 1, 4, 3, 2, 0, 3, 4, 2, 2, 0, 1]
pred += [10, 46, 35, 13, 29, 8, 15, 31]


100it [07:15,  4.36s/it]

data += [41, 25, 13, 37, 5, 49, 24, 17, 32, 6, 44, 28, 15, 30, 3, 40, 20, 14, 38, 0, 48, 23, 12, 31, 1]
pred_label += [4, 0, 3, 3, 0, 4, 0, 1, 4, 0, 4, 2, 3, 3, 0, 4, 0, 1, 2, 0, 4, 0, 1, 2, 0]
pred += [25, 13, 24, 32, 15, 20, 38, 23, 31]
epoch 10, val, loss=1.8179 acc=0.7168 maxacc=0.7708


ETA:1.9h/1.9h


## Testing

In [30]:
# os.listdir('./save/subspace-5w5sdiscriminativeJantung_final/')

In [31]:
# args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final/efnetb1_doaug-final_2021_10_07_08_06'

In [32]:
print("-----------------------------TESTING-----------------------------")
print("-----------------------------------------------------------------")

from tqdm import tqdm

# LOAD SUPPORT SET
trlog = torch.load(args['save-path']+'/trlog')
support_val_sampler = torch.load(args['save-path']+'/support-set.pt')
val_loader = DataLoader(dataset=valset, batch_sampler=support_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

val_support_data = [] 
val_support_label = [] 
for i, batch in tqdm(enumerate(val_loader, 1)):
    data, _ = [_.cuda() for _ in batch] 
    p = args['shot'] * args['test-way']
    data_shot, data_query = data[:p], data[p:]
    val_support_data.append(data_shot) 
    val_support_label.append(_[:p][:5]) 

# LOAD MODEL
model.load_state_dict(torch.load(osp.join(args['save-path'], 'max-acc.pth')))
model.eval()

trlog['test_data_in_max_acc'] = []
trlog['test_label_in_max_acc'] = []

for i, batch in tqdm(enumerate(test_loader, 1)):
    data, _ = [_.cuda() for _ in batch]

    p = args['shot'] * args['test-way'] 
    data_shot, data_query = data[:p], data[p:]
    data_shot = val_support_data[i-1] 

    if args['shot'] == 1:
        data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

    proto = model(data_shot)
    proto = proto.reshape(shot_num, args['test-way'], -1) 
    proto = torch.transpose(proto, 0, 1)
    hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)

    logits, ___ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
    prediction = torch.argmax(logits, axis=1)
    prediction = [val_support_label[i-1][il.item()].item() for il in prediction]
    
    trlog['test_data_in_max_acc'] += [__.item() for __ in _[p:]] 
    trlog['test_label_in_max_acc'] += prediction

0it [00:00, ?it/s]

-----------------------------TESTING-----------------------------
-----------------------------------------------------------------


100it [07:03,  4.24s/it]
100it [09:02,  5.42s/it]


In [33]:
import pandas as pd

pd.DataFrame({
    'data':trlog['test_data_in_max_acc'],
    'prediction':trlog['test_label_in_max_acc']
    }).to_csv(
        args['save-path']+'/{}-finaltest-epochs{}_valacc{}.csv'.format(
            modelname,
            trlog['max_epoch'],
            int(round(trlog['max_acc'],4)*10000)), 
        index=False)  

## Training

In [34]:
import os
import torch
import argparse
import numpy as np
import os.path as osp
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader

# SET MODEL NAME
# modelname = 'cnn'
# modelname = 'mbnet'
# modelname = 'mbnet_no'
# modelname = 'efnetb1'
# modelname = 'efnetb1_no'
# modelname = 'efnetb0'
# modelname = 'efnetb2'
# modelname = 'efnetb3'
# modelname = 'efnetb1_do'
# modelname = 'efnetb1aug'
modelname = 'efnetb1_doaug'

num_sampler = 100
nslice = 6

txt = str(datetime.now())
txt = '_'.join([modelname+'-final', txt[:4], txt[5:7], txt[8:10], txt[11:13], txt[14:16]])

args = {}
args['max-epoch'] = 10
args['save-epoch'] = 5
args['shot'] = 5
args['query'] = 5
args['train-way'] = 5
args['test-way'] = 5
args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final-shot5/'+txt
args['data-path'] = 'materials'
args['gpu'] = '0'
args['lamb'] = 0.03
args['subspace-dim'] = args['shot']-1
set_gpu(args['gpu'])

# MODEL BUILDER
model = {
    'cnn': ConvNet(),
    'mbnet_no': Mbnet_nopre(),
    'mbnet': Mbnet(),
    'efnetb1_no': Efnet_nopre(ver='b1'),
    'efnetb0': Efnet(ver='b0'),
    'efnetb1': Efnet(ver='b1'),
    'efnetb2': Efnet(ver='b2'),
    'efnetb3': Efnet(ver='b3'),
    'efnetb1_do': Efnet_do(ver='b1'),
}[modelname.replace('aug','')].cuda()


using gpu: 0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b1


In [35]:
# GENERATE RANDOM TRAINING VALIDATION
train = []
test = []
csvpath = 'materials/'
csvfile = [_[:-1] for _ in open(csvpath+'all_random3.csv', 'r').readlines()[1:]]
dcm,hcm,minf,nor,rv = np.arange(0,20), np.arange(20,40), np.arange(40,60), np.arange(60,80), np.arange(80,100)
for _ in [dcm,hcm,minf,nor,rv]:
    perm = list(np.random.permutation(_))
    train += [csvfile[k] for k in perm[:10]] 
    test += [csvfile[k] for k in perm[10:]]
    
test = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in test]
train = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in train]

f =  open(csvpath+'train_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in train[:-1]:
    f.write(wr+'\n')
f.write(train[-1])
f.close()

f =  open(csvpath+'test_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in test[:-1]:
    f.write(wr+'\n')
f.write(test[-1])
f.close()

In [36]:
# DATA LOADER
if 'aug' in modelname:
  trainset = JantungAug('train', args['data-path'])
else:
  trainset = Jantung('train', args['data-path'])
  
train_sampler = CategoriesSampler(trainset.label, num_sampler,
                                  args['train-way'], args['shot'] + args['query'])
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, shuffle=False,)
#                           num_workers=8, pin_memory=True)

valset = Jantung('test', args['data-path'])
val_sampler = CategoriesSampler(valset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])
list_val_sampler = []
for pr in val_sampler:
    list_val_sampler.append([h.item() for h in pr])
list_val_sampler = torch.from_numpy(np.array(list_val_sampler))
val_loader = DataLoader(dataset=valset, batch_sampler=list_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

testset = Jantung('testfinal', args['data-path'])
test_sampler = CategoriesSampler_Custom(testset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])

test_loader = DataLoader(dataset=testset, batch_sampler=test_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)


In [37]:
from tqdm import tqdm
projection_pro = Subspace_Projection(num_dim=args['subspace-dim'])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if args['shot'] > 1:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

if args['shot'] == 1:
    shot_num = 2
    args['subspace-dim'] = 1
else:
    shot_num = args['shot']

In [38]:
# TRAINING PHASE
if not os.path.exists(args['save-path']):
    os.mkdir(args['save-path'])
torch.save(list_val_sampler, args['save-path']+'/support-set.pt')
def save_model(name):
    if not os.path.exists(args['save-path']):
        os.mkdir(args['save-path'])
    torch.save(model.state_dict(), osp.join(args['save-path'], name + '.pth'))

trlog = {}
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0
trlog['max_epoch'] = 0

timer = Timer()

for epoch in range(1, args['max-epoch'] + 1):
    lr_scheduler.step()

    model.train()

    tl = Averager()
    ta = Averager()

    for i, batch in tqdm(enumerate(train_loader, 1)):
        data, _ = [_.cuda() for _ in batch]
        p = args['shot'] * args['train-way']
        qq = p + args['query'] * args['train-way']
        data_shot, data_query = data[:p], data[p:qq]
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['train-way'], -1)
        proto = torch.transpose(proto, 0, 1)
        hyperplanes, mu = projection_pro.create_subspace(proto, args['train-way'], shot_num)

        label = torch.arange(args['train-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, discriminative_loss = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        loss = F.cross_entropy(logits, label) + args['lamb']*discriminative_loss
        acc = count_acc(logits, label)

        tl.add(loss.item())
        ta.add(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch {},   loss={:.4f} acc={:.4f}'
          .format(epoch,     loss.item(), acc))

    tl = tl.item()
    ta = ta.item()

    if epoch < 100 and epoch%2!=0:
        continue
    model.eval()

    vl = Averager()
    va = Averager()

    vk = []
    vs = []
    for pr in list_val_sampler:
        vk.append([valset.label[h.item()] for h in pr])
        vs.append([h.item() for h in pr])
        
    val_support_data = [] 
    val_support_label = [] 
    for i, batch in tqdm(enumerate(val_loader, 1)):
        data, _ = [_.cuda() for _ in batch] 
        
        p = args['shot'] * args['test-way']
        
        data_shot, data_query = data[:p], data[p:]

        val_support_data.append(data_shot) 
        val_support_label.append(_[:p][:5])        
        print('data +=',vs[i-1][p:])
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['test-way'], -1) 
        proto = torch.transpose(proto, 0, 1)
        hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
        
        label = torch.arange(args['test-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        prediction = torch.argmax(logits, axis=1)
        pl = vk[i-1][p:][:5]
        pred_label = [pl[ii.item()] for ii in prediction]
        print('pred_label +=',pred_label)

        pros = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()==label[ii].item()]
        cons = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()!=label[ii].item()]
        print('pred +=',cons)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)

        vl.add(loss.item())
        va.add(acc)

    vl = vl.item()
    va = va.item()
    print('epoch {}, val, loss={:.4f} acc={:.4f} maxacc={:.4f}'.format(epoch, vl, va,trlog['max_acc']))

    if va > trlog['max_acc']:
        trlog['max_acc'] = va
        save_model('max-acc')
        trlog['max_epoch'] = epoch

    trlog['train_loss'].append(tl)
    trlog['train_acc'].append(ta)
    trlog['val_loss'].append(vl)
    trlog['val_acc'].append(va)

    torch.save(trlog, osp.join(args['save-path'], 'trlog'))

    save_model('epoch-last')

    if epoch % args['save-epoch'] == 0:
        save_model('epoch-{}'.format(epoch))

    print('ETA:{}/{}'.format(timer.measure(), timer.measure(epoch / args['max-epoch'])))


100it [07:27,  4.47s/it]
0it [00:00, ?it/s]

epoch 1,   loss=0.3697 acc=0.9200


100it [07:30,  4.50s/it]
0it [00:00, ?it/s]

epoch 2,   loss=0.0969 acc=0.9600


1it [00:04,  4.35s/it]

...

99it [07:09,  4.34s/it]

data += [5, 45, 31, 26, 11, 0, 47, 36, 27, 10, 3, 46, 30, 23, 16, 7, 42, 33, 25, 12, 8, 48, 37, 24, 19]
pred_label += [2, 4, 3, 2, 1, 3, 4, 3, 2, 1, 0, 4, 3, 3, 1, 0, 4, 2, 2, 1, 0, 4, 3, 0, 1]
pred += [5, 0, 23, 33, 24]


100it [07:13,  4.34s/it]

data += [33, 0, 49, 14, 29, 30, 2, 44, 11, 23, 39, 3, 48, 10, 28, 38, 5, 46, 15, 24, 37, 8, 42, 16, 20]
pred_label += [2, 3, 4, 1, 3, 3, 0, 3, 1, 3, 3, 0, 4, 1, 2, 3, 2, 4, 1, 0, 3, 0, 4, 1, 2]
pred += [33, 0, 29, 44, 23, 5, 24]
epoch 10, val, loss=2.0116 acc=0.7056 maxacc=0.7400


ETA:1.8h/1.8h


## Testing

In [39]:
# os.listdir('./save/subspace-5w5sdiscriminativeJantung_final/')

In [40]:
# args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final/efnetb1_doaug-final_2021_10_07_08_06'

In [41]:
print("-----------------------------TESTING-----------------------------")
print("-----------------------------------------------------------------")

from tqdm import tqdm

# LOAD SUPPORT SET
trlog = torch.load(args['save-path']+'/trlog')
support_val_sampler = torch.load(args['save-path']+'/support-set.pt')
val_loader = DataLoader(dataset=valset, batch_sampler=support_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

val_support_data = [] 
val_support_label = [] 
for i, batch in tqdm(enumerate(val_loader, 1)):
    data, _ = [_.cuda() for _ in batch] 
    p = args['shot'] * args['test-way']
    data_shot, data_query = data[:p], data[p:]
    val_support_data.append(data_shot) 
    val_support_label.append(_[:p][:5]) 

# LOAD MODEL
model.load_state_dict(torch.load(osp.join(args['save-path'], 'max-acc.pth')))
model.eval()

trlog['test_data_in_max_acc'] = []
trlog['test_label_in_max_acc'] = []

for i, batch in tqdm(enumerate(test_loader, 1)):
    data, _ = [_.cuda() for _ in batch]

    p = args['shot'] * args['test-way'] 
    data_shot, data_query = data[:p], data[p:]
    data_shot = val_support_data[i-1] 

    if args['shot'] == 1:
        data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

    proto = model(data_shot)
    proto = proto.reshape(shot_num, args['test-way'], -1) 
    proto = torch.transpose(proto, 0, 1)
    hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)

    logits, ___ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
    prediction = torch.argmax(logits, axis=1)
    prediction = [val_support_label[i-1][il.item()].item() for il in prediction]
    
    trlog['test_data_in_max_acc'] += [__.item() for __ in _[p:]] 
    trlog['test_label_in_max_acc'] += prediction

0it [00:00, ?it/s]

-----------------------------TESTING-----------------------------
-----------------------------------------------------------------


100it [07:03,  4.24s/it]
100it [08:59,  5.39s/it]


In [42]:
import pandas as pd

pd.DataFrame({
    'data':trlog['test_data_in_max_acc'],
    'prediction':trlog['test_label_in_max_acc']
    }).to_csv(
        args['save-path']+'/{}-finaltest-epochs{}_valacc{}.csv'.format(
            modelname,
            trlog['max_epoch'],
            int(round(trlog['max_acc'],4)*10000)), 
        index=False)  

## Training

In [43]:
import os
import torch
import argparse
import numpy as np
import os.path as osp
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader

# SET MODEL NAME
# modelname = 'cnn'
# modelname = 'mbnet'
# modelname = 'mbnet_no'
# modelname = 'efnetb1'
# modelname = 'efnetb1_no'
# modelname = 'efnetb0'
# modelname = 'efnetb2'
# modelname = 'efnetb3'
# modelname = 'efnetb1_do'
# modelname = 'efnetb1aug'
modelname = 'efnetb1_doaug'

num_sampler = 100
nslice = 6

txt = str(datetime.now())
txt = '_'.join([modelname+'-final', txt[:4], txt[5:7], txt[8:10], txt[11:13], txt[14:16]])

args = {}
args['max-epoch'] = 10
args['save-epoch'] = 5
args['shot'] = 5
args['query'] = 5
args['train-way'] = 5
args['test-way'] = 5
args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final-shot5/'+txt
args['data-path'] = 'materials'
args['gpu'] = '0'
args['lamb'] = 0.03
args['subspace-dim'] = args['shot']-1
set_gpu(args['gpu'])

# MODEL BUILDER
model = {
    'cnn': ConvNet(),
    'mbnet_no': Mbnet_nopre(),
    'mbnet': Mbnet(),
    'efnetb1_no': Efnet_nopre(ver='b1'),
    'efnetb0': Efnet(ver='b0'),
    'efnetb1': Efnet(ver='b1'),
    'efnetb2': Efnet(ver='b2'),
    'efnetb3': Efnet(ver='b3'),
    'efnetb1_do': Efnet_do(ver='b1'),
}[modelname.replace('aug','')].cuda()


using gpu: 0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b1


In [44]:
# GENERATE RANDOM TRAINING VALIDATION
train = []
test = []
csvpath = 'materials/'
csvfile = [_[:-1] for _ in open(csvpath+'all_random3.csv', 'r').readlines()[1:]]
dcm,hcm,minf,nor,rv = np.arange(0,20), np.arange(20,40), np.arange(40,60), np.arange(60,80), np.arange(80,100)
for _ in [dcm,hcm,minf,nor,rv]:
    perm = list(np.random.permutation(_))
    train += [csvfile[k] for k in perm[:10]] 
    test += [csvfile[k] for k in perm[10:]]
    
test = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in test]
train = [','.join(_.split(',')[2*(6-nslice):-1]+_.split(',')[-1:]) for _ in train]

f =  open(csvpath+'train_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in train[:-1]:
    f.write(wr+'\n')
f.write(train[-1])
f.close()

f =  open(csvpath+'test_jantung_random3.csv', 'w+')
f.write('name1,name2,Group'+'\n')
for wr in test[:-1]:
    f.write(wr+'\n')
f.write(test[-1])
f.close()

In [45]:
# DATA LOADER
if 'aug' in modelname:
  trainset = JantungAug('train', args['data-path'])
else:
  trainset = Jantung('train', args['data-path'])
  
train_sampler = CategoriesSampler(trainset.label, num_sampler,
                                  args['train-way'], args['shot'] + args['query'])
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, shuffle=False,)
#                           num_workers=8, pin_memory=True)

valset = Jantung('test', args['data-path'])
val_sampler = CategoriesSampler(valset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])
list_val_sampler = []
for pr in val_sampler:
    list_val_sampler.append([h.item() for h in pr])
list_val_sampler = torch.from_numpy(np.array(list_val_sampler))
val_loader = DataLoader(dataset=valset, batch_sampler=list_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

testset = Jantung('testfinal', args['data-path'])
test_sampler = CategoriesSampler_Custom(testset.label, num_sampler,
                                args['test-way'], args['shot'] + args['query'])

test_loader = DataLoader(dataset=testset, batch_sampler=test_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)


In [46]:
from tqdm import tqdm
projection_pro = Subspace_Projection(num_dim=args['subspace-dim'])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if args['shot'] > 1:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

if args['shot'] == 1:
    shot_num = 2
    args['subspace-dim'] = 1
else:
    shot_num = args['shot']

In [47]:
# TRAINING PHASE
if not os.path.exists(args['save-path']):
    os.mkdir(args['save-path'])
torch.save(list_val_sampler, args['save-path']+'/support-set.pt')
def save_model(name):
    if not os.path.exists(args['save-path']):
        os.mkdir(args['save-path'])
    torch.save(model.state_dict(), osp.join(args['save-path'], name + '.pth'))

trlog = {}
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0
trlog['max_epoch'] = 0

timer = Timer()

for epoch in range(1, args['max-epoch'] + 1):
    lr_scheduler.step()

    model.train()

    tl = Averager()
    ta = Averager()

    for i, batch in tqdm(enumerate(train_loader, 1)):
        data, _ = [_.cuda() for _ in batch]
        p = args['shot'] * args['train-way']
        qq = p + args['query'] * args['train-way']
        data_shot, data_query = data[:p], data[p:qq]
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['train-way'], -1)
        proto = torch.transpose(proto, 0, 1)
        hyperplanes, mu = projection_pro.create_subspace(proto, args['train-way'], shot_num)

        label = torch.arange(args['train-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, discriminative_loss = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        loss = F.cross_entropy(logits, label) + args['lamb']*discriminative_loss
        acc = count_acc(logits, label)

        tl.add(loss.item())
        ta.add(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch {},   loss={:.4f} acc={:.4f}'
          .format(epoch,     loss.item(), acc))

    tl = tl.item()
    ta = ta.item()

    if epoch < 100 and epoch%2!=0:
        continue
    model.eval()

    vl = Averager()
    va = Averager()

    vk = []
    vs = []
    for pr in list_val_sampler:
        vk.append([valset.label[h.item()] for h in pr])
        vs.append([h.item() for h in pr])
        
    val_support_data = [] 
    val_support_label = [] 
    for i, batch in tqdm(enumerate(val_loader, 1)):
        data, _ = [_.cuda() for _ in batch] 
        
        p = args['shot'] * args['test-way']
        
        data_shot, data_query = data[:p], data[p:]

        val_support_data.append(data_shot) 
        val_support_label.append(_[:p][:5])        
        print('data +=',vs[i-1][p:])
        
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['test-way'], -1) 
        proto = torch.transpose(proto, 0, 1)
        hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
        
        label = torch.arange(args['test-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)

        logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        prediction = torch.argmax(logits, axis=1)
        pl = vk[i-1][p:][:5]
        pred_label = [pl[ii.item()] for ii in prediction]
        print('pred_label +=',pred_label)

        pros = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()==label[ii].item()]
        cons = [vs[i-1][p:][ii] for ii in range(len(vs[i-1][p:])) if prediction[ii].item()!=label[ii].item()]
        print('pred +=',cons)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)

        vl.add(loss.item())
        va.add(acc)

    vl = vl.item()
    va = va.item()
    print('epoch {}, val, loss={:.4f} acc={:.4f} maxacc={:.4f}'.format(epoch, vl, va,trlog['max_acc']))

    if va > trlog['max_acc']:
        trlog['max_acc'] = va
        save_model('max-acc')
        trlog['max_epoch'] = epoch

    trlog['train_loss'].append(tl)
    trlog['train_acc'].append(ta)
    trlog['val_loss'].append(vl)
    trlog['val_acc'].append(va)

    torch.save(trlog, osp.join(args['save-path'], 'trlog'))

    save_model('epoch-last')

    if epoch % args['save-epoch'] == 0:
        save_model('epoch-{}'.format(epoch))

    print('ETA:{}/{}'.format(timer.measure(), timer.measure(epoch / args['max-epoch'])))


100it [07:30,  4.50s/it]
0it [00:00, ?it/s]

epoch 1,   loss=0.1018 acc=1.0000


100it [07:30,  4.51s/it]
0it [00:00, ?it/s]

epoch 2,   loss=0.0214 acc=1.0000


1it [00:04,  4.37s/it]

...

99it [07:13,  4.41s/it]

data += [47, 4, 33, 22, 10, 46, 7, 30, 26, 14, 49, 1, 36, 24, 19, 48, 6, 32, 20, 12, 42, 0, 39, 29, 11]
pred_label += [4, 2, 4, 2, 1, 4, 0, 4, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 3, 1, 4, 0, 3, 3, 3]
pred += [4, 33, 30, 20, 29, 11]


100it [07:17,  4.38s/it]

data += [6, 33, 25, 18, 40, 7, 37, 26, 19, 49, 1, 34, 23, 12, 45, 0, 31, 20, 17, 44, 2, 36, 21, 15, 48]
pred_label += [0, 3, 0, 1, 4, 0, 3, 2, 1, 4, 0, 3, 3, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 3]
pred += [25, 23, 48]
epoch 10, val, loss=1.5963 acc=0.7480 maxacc=0.7172


ETA:1.9h/1.9h


## Testing

In [48]:
# os.listdir('./save/subspace-5w5sdiscriminativeJantung_final/')

In [49]:
# args['save-path'] = './save/subspace-5w5sdiscriminativeJantung_final/efnetb1_doaug-final_2021_10_07_08_06'

In [50]:
print("-----------------------------TESTING-----------------------------")
print("-----------------------------------------------------------------")

from tqdm import tqdm

# LOAD SUPPORT SET
trlog = torch.load(args['save-path']+'/trlog')
support_val_sampler = torch.load(args['save-path']+'/support-set.pt')
val_loader = DataLoader(dataset=valset, batch_sampler=support_val_sampler, shuffle=False,)
#                         num_workers=8, pin_memory=True)

val_support_data = [] 
val_support_label = [] 
for i, batch in tqdm(enumerate(val_loader, 1)):
    data, _ = [_.cuda() for _ in batch] 
    p = args['shot'] * args['test-way']
    data_shot, data_query = data[:p], data[p:]
    val_support_data.append(data_shot) 
    val_support_label.append(_[:p][:5]) 

# LOAD MODEL
model.load_state_dict(torch.load(osp.join(args['save-path'], 'max-acc.pth')))
model.eval()

trlog['test_data_in_max_acc'] = []
trlog['test_label_in_max_acc'] = []

for i, batch in tqdm(enumerate(test_loader, 1)):
    data, _ = [_.cuda() for _ in batch]

    p = args['shot'] * args['test-way'] 
    data_shot, data_query = data[:p], data[p:]
    data_shot = val_support_data[i-1] 

    if args['shot'] == 1:
        data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

    proto = model(data_shot)
    proto = proto.reshape(shot_num, args['test-way'], -1) 
    proto = torch.transpose(proto, 0, 1)
    hyperplanes,  mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)

    logits, ___ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
    prediction = torch.argmax(logits, axis=1)
    prediction = [val_support_label[i-1][il.item()].item() for il in prediction]
    
    trlog['test_data_in_max_acc'] += [__.item() for __ in _[p:]] 
    trlog['test_label_in_max_acc'] += prediction

0it [00:00, ?it/s]

-----------------------------TESTING-----------------------------
-----------------------------------------------------------------


100it [07:07,  4.27s/it]
100it [09:02,  5.43s/it]


In [51]:
import pandas as pd

pd.DataFrame({
    'data':trlog['test_data_in_max_acc'],
    'prediction':trlog['test_label_in_max_acc']
    }).to_csv(
        args['save-path']+'/{}-finaltest-epochs{}_valacc{}.csv'.format(
            modelname,
            trlog['max_epoch'],
            int(round(trlog['max_acc'],4)*10000)), 
        index=False)  